# Acartia Sightings Mapped for Correlation with Sunset Bay Hydrophone and Whidbey Tel DAS Data

## Isabelle Brandicourt, 11-10-2024

### Step 1: Map the locations of the sunset bay hydrophone and the DAS cable coordinates we were given

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import numpy as np
import pandas as pd

import geopandas as gpd
import contextily as ctx
from shapely.geometry import box
from scipy.ndimage import zoom

import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LightSource
import matplotlib.colors as mcolors

import rasterio
import rasterio.plot
from rasterio.transform import from_origin
from rasterio.windows import Window
from scipy.interpolate import interp1d
from matplotlib.colors import LinearSegmentedColormap


In [2]:
def plot_all_cables(bbox, clinton_ev, clinton_hat, whidbey_sea, bathy, lats, longs, use, sights, detects, matched_all_entries, matched_s, title):
    seattle = [47.6061, -122.3328]
    sb_hydrophone = [47.865395, -122.33268]
    xlon, ylat = np.meshgrid(np.linspace(bbox[0], bbox[2], 100), np.linspace(bbox[1], bbox[3], 100))
    
    mode9_cablelats = [47.938588224, 47.939247329, 47.939086368, 47.938112322, 47.93311287, 47.913462193, 47.886871002, 47.85318389231053]  
    mode9_cablelongs = [-122.443303496, -122.445259957, -122.447750044, -122.449542984, -122.456240323, -122.455741775, -122.44756012, -122.44188688856443]
    postmode9_sights = sights[sights['date_pstpdt'] >= '2024-10-24']
    
    vmin = -250
    vmax = 100
    
    # Calculate the percentage of the colormap that should be water vs land
    total_range = vmax - vmin
    water_range = 0 - vmin  # from vmin to 0
    water_fraction = water_range / total_range
    
    # Custom colormap, combine colors for water and land
    n_water = int(100 * water_fraction)
    n_land = int(100 * (1 - water_fraction))
    water_colors = plt.cm.Blues(np.linspace(1, 0.2, n_water))
    land_colors = np.linspace([1, 1, 1, 1], [0.7, 0.7, 0.7, 1], n_land)
    all_colors = np.vstack((water_colors, land_colors))
    custom_cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', all_colors, N=256)
    
    # Create BoundaryNorm to force the transition at 0
    bounds = np.linspace(vmin, vmax, 256)
    norm = mcolors.BoundaryNorm(bounds, custom_cmap.N)

    # Set the extent of the plot and light source
    extent = [xlon[0], xlon[-1], ylat[0], ylat[-1]]
    ls = LightSource(azdeg=350, altdeg=45)
    
    plt.figure(figsize=(7, 7))
    ax = plt.gca()
    # rgb = ls.shade(bathy, cmap=custom_cmap, vert_exag=0.1, blend_mode='overlay', norm=norm)
    # plot = ax.imshow(rgb, extent=extent, aspect='equal', origin='lower')

    print(bathy)
    depth = np.where(bathy < 0, bathy, np.nan)
    elevation = np.where(bathy >= 0, bathy, np.nan)

    cp_depth = plt.contourf(xlon, ylat, depth, levels=18, cmap="bone")
    cbar_depth = plt.colorbar(cp_depth, label="Depth (m)")
    cp_elevation = plt.contourf(xlon, ylat, elevation, levels=20, cmap="copper", alpha=0.6)

    ax.plot(clinton_hat["Long"], clinton_hat["Lat"], color="green", linewidth=3, label="Clinton-Hat Cable, 6 km")
    ax.plot(clinton_ev["Long"], clinton_ev["Lat"], color="black", linewidth=3, label="Clinton-Everett Cable, 11 km")
    ax.plot(whidbey_sea["Long"], whidbey_sea["Lat"], color="m", linewidth=3, label="Whidbey-Seattle Cable, 44 km")
    ax.plot(mode9_cablelongs, mode9_cablelats, color='red', linestyle='-', linewidth=3, label='Mode9 Section')
    ax.plot(seattle[1], seattle[0], marker='*', color='red', markersize=15)
    ax.plot(sb_hydrophone[1], sb_hydrophone[0], marker='*', color='red', markersize=15)
    ax.scatter(postmode9_sights['longitude'], postmode9_sights['latitude'], label='Sightings in Mode9', s=8, color='darkmagenta')
    ax.contour(bathy, levels=[0], colors='k', extent=extent)

    #im = ax.imshow(bathy, cmap=custom_cmap, extent=extent, aspect='equal', origin='lower', norm=norm)
    if use == 1:
        plt.xlim(longs[0], longs[1])
        plt.ylim(lats[0], lats[1])
        cbar = plt.colorbar(cbar_depth, ax=ax, label='Depth [m]', aspect=55, pad=0.2, orientation='horizontal')
        cbar.set_label('Depth [m]', fontsize=14, rotation=0, labelpad=20)
    else:
        cbar = plt.colorbar(cbar_depth, ax=ax, label='Depth [m]', aspect=55, pad=0.1, orientation='vertical')
        cbar.set_label('Depth [m]', fontsize=14, rotation=90, labelpad=20)
    cbar.set_ticks([-250, -200, -150, -100, -50, 0, 50, 100])
    cbar.ax.tick_params(labelsize=12)
    #im.remove()
        
    plt.subplots_adjust(bottom=0.0, top=1, left=0.0, right=1)
    plt.title(title, fontsize=20)
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)
    plt.legend(loc='lower left', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return

def srkw_sightings(lat, long):
    sightings = pd.read_csv('/home/isabrand/Documents/ODL/Bathymetry/all_acartia_sightings.csv')
    sightings = sightings[sightings['latitude'] > lat[0]]
    sightings = sightings[sightings['latitude'] < lat[1]]
    sightings = sightings[sightings['longitude'] > long[0]]
    sightings = sightings[sightings['longitude'] < long[1]]
    sightings['date_time_utc'] = pd.to_datetime(sightings['date_time_utc'], format='%Y-%m-%d_%H.%M.%S')
    
    return sightings

def sunset_bay_detections():
    detects = pd.read_csv('/home/isabrand/Documents/ODL/Whidbey/Hydrophone/orcasound_entries.csv')
    detects['date_time_utc'] = pd.to_datetime(detects['date_time_utc'], format='%Y-%m-%d_%H.%M.%S')

    return detects

def matching_sights_sounds(arg_a, arg_b, mins):
    # makes array of arg_b that falls into the time window of one entry in arg_a
    time_window = pd.Timedelta(minutes=mins)
    matching_rows = []
    
    for index, a_row in arg_a.iterrows():
        a_time = a_row['date_time_utc']
        matches = arg_b[
            (arg_b['date_time_utc'] >= a_time - time_window) & 
            (arg_b['date_time_utc'] <= a_time + time_window)
        ].copy()
        matches['latitude'] = a_row['latitude']
        matches['longitude'] = a_row['longitude']
        matching_rows.append(matches)

    result = pd.concat(matching_rows)
    result['date_time_utc'] = pd.to_datetime(result['date_time_utc'], format='%Y-%m-%d_%H.%M.%S')
    result['date_time_pstpdt'] = pd.to_datetime(result['date_time_pstpdt'], format='%Y-%m-%d_%H.%M.%S')
    result_avg_loc = result.copy()
    
    result_avg_loc = result_avg_loc.groupby('date_time_utc', as_index=False).apply(
        lambda group: pd.Series({
            **group.iloc[0].to_dict(),  # Keep the first row's values for all other columns
            'latitude': group['latitude'].mean(),  # Average lat
            'longitude': group['longitude'].mean()  # Average lon
        })
    )
    
    # Convert date_time_pstpdt back to the original format
    result['date_time_utc'] = result['date_time_utc'].dt.strftime('%Y-%m-%d_%H.%M.%S')
    result_avg_loc['date_time_utc'] = result_avg_loc['date_time_utc'].dt.strftime('%Y-%m-%d_%H.%M.%S')
    result['date_time_pstpdt'] = result['date_time_pstpdt'].dt.strftime('%Y-%m-%d_%H.%M.%S')
    result_avg_loc['date_time_pstpdt'] = result_avg_loc['date_time_pstpdt'].dt.strftime('%Y-%m-%d_%H.%M.%S')

    return result, result_avg_loc

In [3]:
sights = srkw_sightings(lat_lims, long_lims)
detects = sunset_bay_detections()

minutes = 60
use = 0
matched_all_entries, matched_s = matching_sights_sounds(sights, detects, minutes)

NameError: name 'lat_lims' is not defined

Plot the entire Whidbey Tel cable, the Mode9 section, and all sightings from Acartia.io after Mode9 was illuminated Then crop to the lat and lon of interest and filter the sightings by time around a detection on the hydrophone.

In [ ]:
clinton_hat = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Mapping_Sam/Clinton-Hat_cable_coordinates.csv")
clinton_ev = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Mapping_Sam/Clinton-Everett_cable_coordinates.csv")
whidbey_sea = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Mapping_Sam/Whidbey-Seattle_cable_coordinates.csv")
bathymetry_file = "/home/isabrand/Documents/ODL/Bathymetry/PugetSound.tiff"

lat_lims = [47.6, 48.2]
long_lims = [-122.5, -122.2]

bbox = [-122.5, 47.6, -122.2, 48.2]
with rasterio.open(bathymetry_file) as src:
    # Get raster bounds
    bounds = src.bounds
    # Check if the bbox intersects the raster
    if (bbox[0] > bounds.right or bbox[2] < bounds.left or 
        bbox[1] > bounds.top or bbox[3] < bounds.bottom):
        raise ValueError("Bounding box outside raster extent.")

    # Read data within the bbox
    window = src.window(*bbox)
    data = src.read(1, window=window)
    transform = src.window_transform(window)

    # Extract coordinates and values
    rows, cols = np.where(~np.isnan(data))
    lats, lons, depths = [], [], []
    for row, col in zip(rows, cols):
        lon, lat = transform * (col, row)
        if bbox[0] <= lon <= bbox[2] and bbox[1] <= lat <= bbox[3]:
            depths.append(data[row, col])
            lons.append(lon)
            lats.append(lat)

plot_all_cables(bbox, clinton_ev, clinton_hat, whidbey_sea, depths, lats, lons, use, sights, detects, matched_all_entries, matched_s, title = 'All Acartia Detections')

use=1
lat = [47.84, 47.95]
long = [-122.55, -122.3]
#plot_all_cables(clinton_ev, clinton_hat, whidbey_sea, bathy, lat, long, use, lon_grid, lat_grid, sights, title='Detections with Sightings within 30 min')

sights = srkw_sightings(lat_lims, long_lims)
detects = sunset_bay_detections()
matched_all_entries, matched_s = matching_sights_sounds(sights, detects, minutes)

print("Number of detections that fall into the time window of a sighting:", len(matched_s))

#matched_s.to_csv("matched_detects_to_visuals.csv", index=False)
#atched_all_entries.to_csv("matched_detections_all_entries.csv", index=False)


### List all of the things I need to plot and the map

In [ ]:
clinton_hat = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Clinton_Hat_cablepath.csv")
clinton_ev = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Clinton_Everett_cablepath.csv")
whidbey_sea = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Whidbey_Seattle_cablepath.csv")
whidbey_sea_mode9 = pd.read_csv("/home/isabrand/Documents/ODL/Bathymetry/Whidbey_Seattle_Mode9_cablepath.csv")

seattle = [47.6061, -122.3328]
sb_hydrophone = [47.865395, -122.33268]

pugetsound_tiff = '/home/isabrand/Documents/ODL/Bathymetry/exportImage.tiff'

puget_sound_lats = [47.8, 48.06]
puget_sound_longs = [-122.535, -122.182]

to_plot = [clinton_ev, clinton_hat, whidbey_sea, whidbey_sea_mode9]


### Functions

In [ ]:
from rasterio.transform import Affine

def nc_to_bathy(nc_file_path):
    # Open NetCDF file
    ds = xr.open_dataset(nc_file_path)
    
    # Extract data (using 'Band1' as elevation/depth)
    bathy = ds['Band1'].values
    
    # Get coordinates
    x = ds['x'].values  # Typically easting in meters
    y = ds['y'].values  # Typically northing in meters
    
    # Create affine transform for transverse mercator projection
    dx = x[1] - x[0]  # Pixel width
    dy = y[1] - y[0]  # Pixel height
    transform = Affine.translation(x[0] - dx/2, y[0] - dy/2) * Affine.scale(dx, dy)
    
    # Get CRS from transverse_mercator attributes
    tm = ds['transverse_mercator']
    crs = rasterio.crs.CRS.from_proj4(
        f"+proj=tmerc +lat_0={tm.latitude_of_projection_origin} "
        f"+lon_0={tm.longitude_of_central_meridian} "
        f"+k={tm.scale_factor_at_central_meridian} "
        f"+x_0={tm.false_easting} +y_0={tm.false_northing} "
        f"+ellps={tm.ellipsoid if 'ellipsoid' in tm.attrs else 'WGS84'}"
    )
    
    # Ensure water depths are negative and land is positive
    bathy = np.where(bathy > 0, -bathy, abs(bathy))
    
    return bathy, x, y, transform, crs


def bathy_load(filepath):
    with rasterio.open(filepath) as src:
        bathy = src.read(1)  # Read the first band (bathymetry data)
        print(bathy)
        transform = src.transform
        crs = src.crs
        height, width = bathy.shape

        # Generate longitude and latitude arrays
        xlon = np.linspace(transform[2], transform[2] + transform[0] * width, width)
        ylat = np.linspace(transform[5], transform[5] + transform[4] * height, height)

    return bathy, xlon, ylat, transform, crs


def plot_map(to_plot, bathy, lat, long, xlon, ylat, title):  
    vmin = -200
    vmax = 200
    
    clinton_ev, clinton_hat, whidbey_sea, whidbey_sea_mode9_cablelats, whidbey_sea_mode9_cablelongs = to_plot[0], to_plot[1], to_plot[2], to_plot[3], to_plot[4]
    
    # Prepare data
    depth = np.where(bathy < 0, bathy, np.nan)
    elevation = np.where(bathy >= 0, bathy, np.nan)
    
    # Create a single figure
    fig, ax = plt.subplots(figsize=(15, 10), dpi=600)
    
    # Plot depth (underwater)
    cp_depth = ax.contourf(xlon, ylat, bathy, levels=50, cmap="bone", vmin=vmin, vmax=vmax)
    cbar_depth = fig.colorbar(cp_depth, ax=ax, label="Depth (m)", pad=0.1)
    
    # Plot elevation (land)
    #cp_elevation = ax.contourf(xlon, ylat, elevation, levels=20, cmap="copper", vmin=0, vmax=vmax, alpha=0.6)
    #cbar_elevation = fig.colorbar(cp_elevation, ax=ax, label="Elevation (m)", pad=0.15)
    
    # Plot cables and markers
    #ax.plot(whidbey_sea_mode9_cablelongs, whidbey_sea_mode9_cablelats, 'tab:red', linewidth=3, label='Whidbey-Seattle Mode 9')
    #ax.plot(whidbey_sea['Long'], whidbey_sea['Lat'], 'tab:red', linewidth=1, label='Whidbey-Seattle Cable')
    #ax.plot(clinton_hat['Long'], clinton_hat['Lat'], 'tab:purple', linewidth=1, label='Clinton-Hat Cable')
    #ax.plot(clinton_ev['Long'], clinton_ev['Lat'], 'tab:green', linewidth=1, label='Clinton-Everett Cable')
    
    # Plot coastline (0m contour)
    ax.contour(xlon, ylat, bathy, levels=[0], colors='k', linewidths=0.5)
    
    # Plot points (Seattle, hydrophone)
    #ax.plot(seattle[1], seattle[0], marker='*', color='red', markersize=20, label='Seattle')
    #ax.plot(sb_hydrophone[1], sb_hydrophone[0], marker='*', color='red', markersize=20, label='Hydrophone')
    
    # Set plot limits
    ax.set_xlim(long[0], long[1])
    ax.set_ylim(lat[0], lat[1])
    
    # Labels and title
    ax.set_title(title, fontsize=20)
    ax.set_xlabel('Longitude', fontsize=14)
    ax.set_ylabel('Latitude', fontsize=14)
    
    # Legend
    ax.legend(loc='lower left', fontsize=10)
    
    # Adjust layout
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
#bathy, xlon, ylat, transform, crs = bathy_load(pugetsound_tiff)
bathy, x, y, transform, crs = nc_to_bathy('/home/isabrand/Documents/ODL/Bathymetry/puget_sound.nc')
plot_map(to_plot, bathy, puget_sound_lats, puget_sound_longs, x, y, title='Puget Sound Bathymetry')


In [ ]:
import pyproj

transformer = pyproj.Transformer.from_crs(
    crs,
    "EPSG:4326",  # WGS84
    always_xy=True
)

# Convert x,y to lon,lat
xx, yy = np.meshgrid(x, y)
lon, lat = transformer.transform(xx, yy)

plt.figure(figsize=(12, 8))
plt.imshow(bathy, extent=(lon.min(), lon.max(), lat.min(), lat.max()),
           cmap='bone', vmin=-250, vmax=250)
plt.colorbar(label='Elevation (m)')
plt.title('Bathymetry/Topography (Transverse Mercator)')
plt.xlabel('Longitude (m)')
plt.ylabel('Latitude (m)')
plt.show()